In [1]:
# Clonar el repositorio (solo se hace una vez)
!rm -rf objetos_salon
!git clone https://github.com/Gerardo-cursos/objetos_salon.git


Cloning into 'objetos_salon'...
remote: Enumerating objects: 2822, done.
remote: Counting objects: 100% (261/261), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 2822 (delta 232), reused 185 (delta 185), pack-reused 2561 (from 2)
Receiving objects: 100% (2822/2822), 1.31 GiB | 30.82 MiB/s, done.
Resolving deltas: 100% (555/555), done.
Updating files: 100% (1959/1959), done.


In [2]:
import os

base_path = "/content/objetos_salon/processed"

print("Carpetas encontradas:", os.listdir(base_path))


Carpetas encontradas: ['cpu', 'README.md', 'nada', 'mouse', 'teclado', 'silla', 'pantalla', 'mesa']


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models


In [4]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    base_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True,
    subset="training"
)

val_generator = datagen.flow_from_directory(
    base_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False,
    subset="validation"
)


Found 1450 images belonging to 7 classes.
Found 361 images belonging to 7 classes.


In [5]:
num_clases = len(train_generator.class_indices)
print("Clases detectadas:", train_generator.class_indices)

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_clases, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Clases detectadas: {'cpu': 0, 'mesa': 1, 'mouse': 2, 'nada': 3, 'pantalla': 4, 'silla': 5, 'teclado': 6}


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    22,151,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,246,471 (84.86 MB)

 Trainable params: 22,246,471 (84.86 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 200s 4s/step - accuracy: 0.3116 - loss: 2.3187 - val_accuracy: 0.5762 - val_loss: 1.5407
Epoch 2/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 204s 4s/step - accuracy: 0.8402 - loss: 0.4695 - val_accuracy: 0.6676 - val_loss: 1.2024
Epoch 3/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 259s 4s/step - accuracy: 0.9340 - loss: 0.1936 - val_accuracy: 0.7341 - val_loss: 1.2274
Epoch 4/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 202s 4s/step - accuracy: 0.9507 - loss: 0.1303 - val_accuracy: 0.7341 - val_loss: 1.2851
Epoch 5/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 208s 5s/step - accuracy: 0.9595 - loss: 0.0909 - val_accuracy: 0.7230 - val_loss: 1.4795
Epoch 6/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 204s 4s/step - accuracy: 0.9634 - loss: 0.0774 - val_accuracy: 0.7036 - val_loss: 1.4439
Epoch 7/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 227s 5s/step - accuracy: 0.9779 - loss: 0.0462 - val_accuracy: 0.7285 - val_loss: 1.2742
Epoch 8/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 201s 4s/step - accuracy: 0.9607 - loss: 0.0667 - val_accuracy: 0.7285 - v

In [7]:
model.save("modelo_objetos_salon.h5")
print("Modelo guardado ✔")

Modelo guardado ✔


In [9]:
from tensorflow.keras.models import load_model
model = load_model("modelo_objetos_salon.h5")

In [13]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Obtener las clases en el orden correcto
class_indices_dict = train_generator.class_indices
clases = [None] * len(class_indices_dict)
for class_name, index in class_indices_dict.items():
    clases[index] = class_name

def predecir_imagen(ruta):
    img = image.load_img(ruta, target_size=(224,224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    pred = model.predict(img_array)[0]
    indice = np.argmax(pred)
    prob = pred[indice]

    print("Predicción:", clases[indice])
    print("Probabilidad:", f"{prob*100:.2f}%")

In [15]:
ruta = "/content/objetos_salon/processed/teclado/20172167090_Teclado_1.png"
predecir_imagen(ruta)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Predicción: teclado
Probabilidad: 99.99%
